In [29]:
user_data_path = "/user/n.mokrenko/data_1/trainDemography"
country_data_path = "/user/n.mokrenko/data_1/geography/countries.csv"
current_dt = "2019-05-01"
output_path = "/user/n.mokrenko/hometask_1"

In [30]:
from pyspark.sql.types import *
import pyspark.sql.functions as sf
from datetime import datetime

### Переводим current_dt в unix time

In [31]:
current_dt_unix = datetime.strptime(str(current_dt), "%Y-%m-%d")
unix_time = datetime.strptime("01.01.1970", "%d.%m.%Y")
cur_dt = ((current_dt_unix - unix_time).days)

### Создаем два датафрейма из countries.csv и trainDemography

In [32]:
cntr_df = spark.read.csv(country_data_path).toDF('ID_country', 'name_country')
df = spark.read.csv(user_data_path, header=False, sep = '\t').toDF(
    'userId','create_date','birth_date','gender','ID_country','ID_location', 'loginRegion', 'isInGraph') # задаем названия колонок


### Создаем новый датафрейм, исходя из требований к заданию

In [33]:
cnt_cond = lambda cond: sf.sum(sf.when(cond, 1).otherwise(0)) # функция проверки соответсвия значения ячейки значению cond
share_cond = lambda cond: sf.round(sf.sum(sf.when(cond, 1).otherwise(0))/sf.count('gender'), 2) # функция для подсчета доли различных гендеров в столбце gender
new_df = df.join(cntr_df, df.ID_country == cntr_df.ID_country, 'left').withColumn( # присоединяем таблицу-справочник countries по id страны
    'age', sf.round((cur_dt - df.birth_date)/365.25)).groupBy("name_country").agg( # добавляем столбец "age", в который записываем разницу между current_dt и датой рождения заказчика   
    sf.round(sf.avg('age'), 2).alias('age_avg'), # считаем средний возраст заказчика
    sf.count('userId').alias('usr_cnt'), # считаем количество пользователей
    cnt_cond(sf.col('gender') == 1).alias('men_cnt'), # считаем количество пользователей - мужчин
    cnt_cond(sf.col('gender') == 2).alias('women_cnt'), # считаем количество пользователей - женщин
    share_cond(sf.col('gender') == 1).alias('men_share'), # считаем долю мужчин
    share_cond(sf.col('gender') == 2).alias('women_share') # считаем долю женщин
).orderBy(sf.desc('usr_cnt')) # сортируем по количеству пользователей в порядке убывания

### Создаем csv с нашим окончательным датафреймом и записываем его в output path

In [24]:
new_df.sortWithinPartitions(sf.desc('usr_cnt')).coalesce(1).write.csv(path=output_path, mode="overwrite", sep="\t", header=True)

### Проверка

In [25]:
q = spark.read.csv(output_path, header=True, sep = '\t').show(10)

+------------+-------+-------+-------+---------+---------+-----------+
|name_country|age_avg|usr_cnt|men_cnt|women_cnt|men_share|women_share|
+------------+-------+-------+-------+---------+---------+-----------+
|      Россия|  37.36|3335808|1252716|  2083092|     0.38|       0.62|
|     Украина|  38.43| 843680| 252132|   591548|      0.3|        0.7|
|  Узбекистан|  29.46| 309207| 229318|    79889|     0.74|       0.26|
|     Молдова|  34.08| 195608|  85112|   110496|     0.44|       0.56|
|    Беларусь|  36.66| 152167|  46828|   105339|     0.31|       0.69|
|   Казахстан|  38.52| 147144|  46653|   100491|     0.32|       0.68|
| Азербайджан|  31.14| 130537| 110039|    20498|     0.84|       0.16|
|      Грузия|  36.34| 118147|  64222|    53925|     0.54|       0.46|
| Таджикистан|  28.93|  84599|  70083|    14516|     0.83|       0.17|
|  Кыргызстан|  29.61|  83021|  39998|    43023|     0.48|       0.52|
+------------+-------+-------+-------+---------+---------+-----------+
only s